https://www.kaggle.com/c/trends-assessment-prediction/overview

In [2]:
import pandas as pd
from lightgbm import  LGBMRegressor
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error

In [3]:
from sklearn.model_selection import KFold

In [4]:
pd.options.display.max_columns = 500

In [5]:
fnc = pd.read_csv('fnc.csv')
loading = pd.read_csv('loading.csv')

In [127]:
fnc

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),SMN(27)_vs_SCN(69),SMN(54)_vs_SCN(69),SMN(66)_vs_SCN(69),SMN(80)_vs_SCN(69),SMN(72)_vs_SCN(69),VSN(16)_vs_SCN(69),VSN(5)_vs_SCN(69),VSN(62)_vs_SCN(69),VSN(15)_vs_SCN(69),VSN(12)_vs_SCN(69),VSN(93)_vs_SCN(69),VSN(20)_vs_SCN(69),VSN(8)_vs_SCN(69),VSN(77)_vs_SCN(69),CON(68)_vs_SCN(69),CON(33)_vs_SCN(69),CON(43)_vs_SCN(69),CON(70)_vs_SCN(69),CON(61)_vs_SCN(69),CON(55)_vs_SCN(69),CON(63)_vs_SCN(69),CON(79)_vs_SCN(69),CON(84)_vs_SCN(69),CON(96)_vs_SCN(69),CON(88)_vs_SCN(69),CON(48)_vs_SCN(69),CON(81)_vs_SCN(69),CON(37)_vs_SCN(69),CON(67)_vs_SCN(69),CON(38)_vs_SCN(69),CON(83)_vs_SCN(69),DMN(32)_vs_SCN(69),DMN(40)_vs_SCN(69),DMN(23)_vs_SCN(69),DMN(71)_vs_SCN(69),DMN(17)_vs_SCN(69),DMN(51)_vs_SCN(69),DMN(94)_vs_SCN(69),CBN(13)_vs_SCN(69),CBN(18)_vs_SCN(69),CBN(4)_vs_SCN(69),CBN(7)_vs_SCN(69),SCN(98)_vs_SCN(53),SCN(99)_vs_SCN(53),SCN(45)_vs_SCN(53),ADN(21)_vs_SCN(53),ADN(56)_vs_SCN(53),SMN(3)_vs_SCN(53),SMN(9)_vs_SCN(53),SMN(2)_vs_SCN(53),SMN(11)_vs_SCN(53),SMN(27)_vs_SCN(53),SMN(54)_vs_SCN(53),SMN(66)_vs_SCN(53),SMN(80)_vs_SCN(53),SMN(72)_vs_SCN(53),VSN(16)_vs_SCN(53),VSN(5)_vs_SCN(53),VSN(62)_vs_SCN(53),VSN(15)_vs_SCN(53),VSN(12)_vs_SCN(53),VSN(93)_vs_SCN(53),VSN(20)_vs_SCN(53),VSN(8)_vs_SCN(53),VSN(77)_vs_SCN(53),CON(68)_vs_SCN(53),CON(33)_vs_SCN(53),CON(43)_vs_SCN(53),CON(70)_vs_SCN(53),CON(61)_vs_SCN(53),CON(55)_vs_SCN(53),CON(63)_vs_SCN(53),CON(79)_vs_SCN(53),CON(84)_vs_SCN(53),CON(96)_vs_SCN(53),CON(88)_vs_SCN(53),CON(48)_vs_SCN(53),CON(81)_vs_SCN(53),CON(37)_vs_SCN(53),CON(67)_vs_SCN(53),CON(38)_vs_SCN(53),CON(83)_vs_SCN(53),DMN(32)_vs_SCN(53),DMN(40)_vs_SCN(53),DMN(23)_vs_SCN(53),DMN(71)_vs_SCN(53),DMN(17)_vs_SCN(53),DMN(51)_vs_SCN(53),DMN(94)_vs_SCN(53),CBN(13)_vs_SCN(53),CBN(18)_vs_SCN(53),CBN(4)_vs_SCN(53),CBN(7)_vs_SCN(53),SCN(99)_vs_SCN(98),SCN(45)_vs_SCN(98),ADN(21)_vs_SCN(98),ADN(56)_vs_SCN(98),SMN(3)_vs_SCN(98),SMN(9)_vs_SCN(98),SMN(2)_vs_SCN(98),SMN(11)_vs_SCN(98),SMN(27)_vs_SCN(98),SMN(54)_vs_SCN(98),SMN(66)_vs_SCN(98),SMN(80)_vs_SCN(98),SMN(72)_vs_SCN(98),VSN(16)_vs_SCN(98),VSN(5)_vs_SCN(98),VSN(62)_vs_SCN(98),VSN(15)_vs_SCN(98),VSN(12)_vs_SCN(98),VSN(93)_vs_SCN(98),VSN(20)_vs_SCN(98),VSN(8)_vs_SCN(98),VSN(77)_vs_SCN(98),CON(68)_vs_SCN(98),CON(33)_vs_SCN(98),CON(43)_vs_SCN(98),CON(70)_vs_SCN(98),CON(61)_vs_SCN(98),CON(55)_vs_SCN(98),CON(63)_vs_SCN(98),CON(79)_vs_SCN(98),CON(84)_vs_SCN(98),CON(96)_vs_SCN(98),CON(88)_vs_SCN(98),CON(48)_vs_SCN(98),CON(81)_vs_SCN(98),CON(37)_vs_SCN(98),CON(67)_vs_SCN(98),CON(38)_vs_SCN(98),CON(83)_vs_SCN(98),DMN(32)_vs_SCN(98),DMN(40)_vs_SCN(98),DMN(23)_vs_SCN(98),DMN(71)_vs_SCN(98),DMN(17)_vs_SCN(98),DMN(51)_vs_SCN(98),DMN(94)_vs_SCN(98),CBN(13)_vs_SCN(98),CBN(18)_vs_SCN(98),CBN(4)_vs_SCN(98),CBN(7)_vs_SCN(98),SCN(45)_vs_SCN(99),ADN(21)_vs_SCN(99),ADN(56)_vs_SCN(99),SMN(3)_vs_SCN(99),SMN(9)_vs_SCN(99),SMN(2)_vs_SCN(99),SMN(11)_vs_SCN(99),SMN(27)_vs_SCN(99),SMN(54)_vs_SCN(99),SMN(66)_vs_SCN(99),SMN(80)_vs_SCN(99),SMN(72)_vs_SCN(99),VSN(16)_vs_SCN(99),VSN(5)_vs_SCN(99),VSN(62)_vs_SCN(99),VSN(15)_vs_SCN(99),VSN(12)_vs_SCN(99),VSN(93)_vs_SCN(99),VSN(20)_vs_SCN(99),VSN(8)_vs_SCN(99),VSN(77)_vs_SCN(99),CON(68)_vs_SCN(99),CON(33)_vs_SCN(99),CON(43)_vs_SCN(99),CON(70)_vs_SCN(99),CON(61)_vs_SCN(99),CON(55)_vs_SCN(99),CON(63)_vs_SCN(99),CON(79)_vs_SCN(99),CON(84)_vs_SCN(99),CON(96)_vs_SCN(99),CON(88)_vs_SCN(99),CON(48)_vs_SCN(99),CON(81)_vs_SCN(99),CON(37)_vs_SCN(99),CON(67)_vs_SCN(99),CON(38)_vs_SCN(99),CON(83)_vs_SCN(99),DMN(32)_vs_SCN(99),DMN(40)_vs_SCN(99),DMN(23)_vs_SCN(99),DMN(71)_vs_SCN(99),DMN(17)_vs_SCN(99),DMN(51)_vs_SCN(99),DMN(94)_vs_SCN(99),CBN(13)_vs_SCN(99),CBN(18)_vs_SCN(99),CBN(4)_vs_SCN(99),CBN(7)_vs_SCN(99),ADN(21)_vs_SCN(45),ADN(56)_vs_SCN(45),SMN(3)_vs_SCN(45),SMN(9)_vs_SCN(45),SMN(2)_vs_SCN(45),SMN(11)_vs_SCN(45),SMN(27)_vs_SCN(45),SMN(54)_vs_SCN(45),SMN(66)_vs_SCN(45),SMN(80)_vs_SCN(45)

In [128]:
loading

,Id,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,IC_04,IC_12,IC_24,IC_15,IC_13,IC_17,IC_02,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,10001,0.006070,0.014466,0.004136,0.000658,-0.002742,0.005033,0.016720,0.003484,0.001797,0.029223,0.007778,0.014609,0.005123,0.017628,0.045102,0.008492,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,10002,0.009087,0.009291,0.007049,-0.002076,-0.002227,0.004605,0.012277,0.002946,0.004086,0.027333,0.005419,0.006232,0.008819,0.017868,0.046214,0.003573,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
2,10003,0.008151,0.014684,0.010444,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,0.038530,0.001721,0.001985,0.012548,0.025707,0.053774,0.011670,0.019565,0.030616,0.018184,-0.010469,0.029799,0.015435,0.005211,-0.028882,0.031427,0.018164
3,10004,0.004675,0.000957,0.006154,-0.000429,-0.001222,0.011755,0.013010,0.000193,0.008075,0.027787,-0.005184,0.004144,0.006837,0.016737,0.045259,0.007893,0.002658,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760
4,10005,-0.000398,0.006878,0.009051,0.000369,0.000336,0.010679,0.010352,0.003637,0.004180,0.021058,0.000377,0.004329,0.005255,0.013587,0.040847,0.008795,0.009702,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,21750,0.005996,0.003873,0.012353,0.000242,-0.002159,0.020201,0.020931,0.003684,-0.002458,0.033895,-0.005588,0.002727,0.012912,0.019439,0.045565,0.013347,0.011454,0.022840,0.006448,-0.007203,0.029689,0.004942,0.007751,-0.020226,0.028821,0.017492
11750,21751,0.003835,0.015067,0.015428,-0.002030,0.001205,0.012396,0.011026,-0.001491,0.005310,0.023853,-0.008467,0.009072,0.020112,0.016211,0.037577,0.006196,0.012303,0.018314,0.012326,-0.012152,0.024617,0.002096,0.001911,-0.008028,0.035093,0.017439
11751,21752,0.000627,0.011407,0.010957,0.000534,-0.000347,0.013499,0.010541,0.001867,0.007447,0.020901,0.000084,0.005250,0.010305,0.017975,0.028226,0.004507,0.012055,0.020741,0.001160,-0.007140,0.025236,0.002026,0.001876,-0.014612,0.021665,0.019592
11752,21753,0.007431,0.021419,0.014143,-0.005623,-0.006399,0.008602,0.006831,-0.001018,0.014972,0.042646,-0.004274,0.002543,0.017471,0.023364,0.054955,0.011054,0.019220,0.026282,0.020715,-0.008130,0.025149,0.007695,0.003226,-0.003505,0.029326,0.014235


In [6]:
train_scores = pd.read_csv('train_scores.csv')

In [129]:
train_scores

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,66.532630,NaN,NaN,52.108977,69.993075
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421
...,...,...,...,...,...,...
5872,21746,14.257265,21.358872,61.165998,51.778483,54.640179
5873,21747,55.456978,68.169675,29.907995,55.349257,54.019517
5874,21750,48.948756,55.114811,60.878271,38.617246,50.679885
5875,21752,66.532630,59.844808,72.303110,55.458281,46.870235


In [7]:
ss = pd.read_csv('sample_submission.csv')

In [130]:
ss

,Id,Predicted
0,10003_age,50.0
1,10003_domain1_var1,50.0
2,10003_domain1_var2,50.0
3,10003_domain2_var1,50.0
4,10003_domain2_var2,50.0
...,...,...
29380,21753_age,50.0
29381,21753_domain1_var1,50.0
29382,21753_domain1_var2,50.0
29383,21753_domain2_var1,50.0


In [8]:
#list(set(ss['Id'].apply(lambda x: int( x.split('_')[0]))))

In [9]:
data = pd.merge(fnc, loading, on='Id', how='left') # Все ИКСЫ, для обучения и тестов.

In [131]:
data

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),SMN(27)_vs_SCN(69),SMN(54)_vs_SCN(69),SMN(66)_vs_SCN(69),SMN(80)_vs_SCN(69),SMN(72)_vs_SCN(69),VSN(16)_vs_SCN(69),VSN(5)_vs_SCN(69),VSN(62)_vs_SCN(69),VSN(15)_vs_SCN(69),VSN(12)_vs_SCN(69),VSN(93)_vs_SCN(69),VSN(20)_vs_SCN(69),VSN(8)_vs_SCN(69),VSN(77)_vs_SCN(69),CON(68)_vs_SCN(69),CON(33)_vs_SCN(69),CON(43)_vs_SCN(69),CON(70)_vs_SCN(69),CON(61)_vs_SCN(69),CON(55)_vs_SCN(69),CON(63)_vs_SCN(69),CON(79)_vs_SCN(69),CON(84)_vs_SCN(69),CON(96)_vs_SCN(69),CON(88)_vs_SCN(69),CON(48)_vs_SCN(69),CON(81)_vs_SCN(69),CON(37)_vs_SCN(69),CON(67)_vs_SCN(69),CON(38)_vs_SCN(69),CON(83)_vs_SCN(69),DMN(32)_vs_SCN(69),DMN(40)_vs_SCN(69),DMN(23)_vs_SCN(69),DMN(71)_vs_SCN(69),DMN(17)_vs_SCN(69),DMN(51)_vs_SCN(69),DMN(94)_vs_SCN(69),CBN(13)_vs_SCN(69),CBN(18)_vs_SCN(69),CBN(4)_vs_SCN(69),CBN(7)_vs_SCN(69),SCN(98)_vs_SCN(53),SCN(99)_vs_SCN(53),SCN(45)_vs_SCN(53),ADN(21)_vs_SCN(53),ADN(56)_vs_SCN(53),SMN(3)_vs_SCN(53),SMN(9)_vs_SCN(53),SMN(2)_vs_SCN(53),SMN(11)_vs_SCN(53),SMN(27)_vs_SCN(53),SMN(54)_vs_SCN(53),SMN(66)_vs_SCN(53),SMN(80)_vs_SCN(53),SMN(72)_vs_SCN(53),VSN(16)_vs_SCN(53),VSN(5)_vs_SCN(53),VSN(62)_vs_SCN(53),VSN(15)_vs_SCN(53),VSN(12)_vs_SCN(53),VSN(93)_vs_SCN(53),VSN(20)_vs_SCN(53),VSN(8)_vs_SCN(53),VSN(77)_vs_SCN(53),CON(68)_vs_SCN(53),CON(33)_vs_SCN(53),CON(43)_vs_SCN(53),CON(70)_vs_SCN(53),CON(61)_vs_SCN(53),CON(55)_vs_SCN(53),CON(63)_vs_SCN(53),CON(79)_vs_SCN(53),CON(84)_vs_SCN(53),CON(96)_vs_SCN(53),CON(88)_vs_SCN(53),CON(48)_vs_SCN(53),CON(81)_vs_SCN(53),CON(37)_vs_SCN(53),CON(67)_vs_SCN(53),CON(38)_vs_SCN(53),CON(83)_vs_SCN(53),DMN(32)_vs_SCN(53),DMN(40)_vs_SCN(53),DMN(23)_vs_SCN(53),DMN(71)_vs_SCN(53),DMN(17)_vs_SCN(53),DMN(51)_vs_SCN(53),DMN(94)_vs_SCN(53),CBN(13)_vs_SCN(53),CBN(18)_vs_SCN(53),CBN(4)_vs_SCN(53),CBN(7)_vs_SCN(53),SCN(99)_vs_SCN(98),SCN(45)_vs_SCN(98),ADN(21)_vs_SCN(98),ADN(56)_vs_SCN(98),SMN(3)_vs_SCN(98),SMN(9)_vs_SCN(98),SMN(2)_vs_SCN(98),SMN(11)_vs_SCN(98),SMN(27)_vs_SCN(98),SMN(54)_vs_SCN(98),SMN(66)_vs_SCN(98),SMN(80)_vs_SCN(98),SMN(72)_vs_SCN(98),VSN(16)_vs_SCN(98),VSN(5)_vs_SCN(98),VSN(62)_vs_SCN(98),VSN(15)_vs_SCN(98),VSN(12)_vs_SCN(98),VSN(93)_vs_SCN(98),VSN(20)_vs_SCN(98),VSN(8)_vs_SCN(98),VSN(77)_vs_SCN(98),CON(68)_vs_SCN(98),CON(33)_vs_SCN(98),CON(43)_vs_SCN(98),CON(70)_vs_SCN(98),CON(61)_vs_SCN(98),CON(55)_vs_SCN(98),CON(63)_vs_SCN(98),CON(79)_vs_SCN(98),CON(84)_vs_SCN(98),CON(96)_vs_SCN(98),CON(88)_vs_SCN(98),CON(48)_vs_SCN(98),CON(81)_vs_SCN(98),CON(37)_vs_SCN(98),CON(67)_vs_SCN(98),CON(38)_vs_SCN(98),CON(83)_vs_SCN(98),DMN(32)_vs_SCN(98),DMN(40)_vs_SCN(98),DMN(23)_vs_SCN(98),DMN(71)_vs_SCN(98),DMN(17)_vs_SCN(98),DMN(51)_vs_SCN(98),DMN(94)_vs_SCN(98),CBN(13)_vs_SCN(98),CBN(18)_vs_SCN(98),CBN(4)_vs_SCN(98),CBN(7)_vs_SCN(98),SCN(45)_vs_SCN(99),ADN(21)_vs_SCN(99),ADN(56)_vs_SCN(99),SMN(3)_vs_SCN(99),SMN(9)_vs_SCN(99),SMN(2)_vs_SCN(99),SMN(11)_vs_SCN(99),SMN(27)_vs_SCN(99),SMN(54)_vs_SCN(99),SMN(66)_vs_SCN(99),SMN(80)_vs_SCN(99),SMN(72)_vs_SCN(99),VSN(16)_vs_SCN(99),VSN(5)_vs_SCN(99),VSN(62)_vs_SCN(99),VSN(15)_vs_SCN(99),VSN(12)_vs_SCN(99),VSN(93)_vs_SCN(99),VSN(20)_vs_SCN(99),VSN(8)_vs_SCN(99),VSN(77)_vs_SCN(99),CON(68)_vs_SCN(99),CON(33)_vs_SCN(99),CON(43)_vs_SCN(99),CON(70)_vs_SCN(99),CON(61)_vs_SCN(99),CON(55)_vs_SCN(99),CON(63)_vs_SCN(99),CON(79)_vs_SCN(99),CON(84)_vs_SCN(99),CON(96)_vs_SCN(99),CON(88)_vs_SCN(99),CON(48)_vs_SCN(99),CON(81)_vs_SCN(99),CON(37)_vs_SCN(99),CON(67)_vs_SCN(99),CON(38)_vs_SCN(99),CON(83)_vs_SCN(99),DMN(32)_vs_SCN(99),DMN(40)_vs_SCN(99),DMN(23)_vs_SCN(99),DMN(71)_vs_SCN(99),DMN(17)_vs_SCN(99),DMN(51)_vs_SCN(99),DMN(94)_vs_SCN(99),CBN(13)_vs_SCN(99),CBN(18)_vs_SCN(99),CBN(4)_vs_SCN(99),CBN(7)_vs_SCN(99),ADN(21)_vs_SCN(45),ADN(56)_vs_SCN(45),SMN(3)_vs_SCN(45),SMN(9)_vs_SCN(45),SMN(2)_vs_SCN(45),SMN(11)_vs_SCN(45),SMN(27)_vs_SCN(45),SMN(54)_vs_SCN(45),SMN(66)_vs_SCN(45),SMN(80)_vs_SCN(45)

In [10]:
test_id = pd.DataFrame(list(set(ss['Id'].apply(lambda x: int( x.split('_')[0])))), columns = ['Id']) # Id тестовых данных

In [11]:
test_id 

,Id
0,10003
1,10006
2,10010
3,10011
4,10012
...,...
5872,21745
5873,21748
5874,21749
5875,21751


In [12]:
def make_test(data, test_id):
    data_X_test = pd.merge(data, test_id, on='Id', how='left')
    #data_y_test = pd.merge(train_scores, test_id, on='Id', how='left')
    # make pd dataframe with Id
    # merge columns  fnc loading
    return data_X_test #, data_y_test

In [13]:
test_data = make_test(data=data, test_id=test_id)

In [14]:
test_data.head()

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),SMN(27)_vs_SCN(69),SMN(54)_vs_SCN(69),SMN(66)_vs_SCN(69),SMN(80)_vs_SCN(69),SMN(72)_vs_SCN(69),VSN(16)_vs_SCN(69),VSN(5)_vs_SCN(69),VSN(62)_vs_SCN(69),VSN(15)_vs_SCN(69),VSN(12)_vs_SCN(69),VSN(93)_vs_SCN(69),VSN(20)_vs_SCN(69),VSN(8)_vs_SCN(69),VSN(77)_vs_SCN(69),CON(68)_vs_SCN(69),CON(33)_vs_SCN(69),CON(43)_vs_SCN(69),CON(70)_vs_SCN(69),CON(61)_vs_SCN(69),CON(55)_vs_SCN(69),CON(63)_vs_SCN(69),CON(79)_vs_SCN(69),CON(84)_vs_SCN(69),CON(96)_vs_SCN(69),CON(88)_vs_SCN(69),CON(48)_vs_SCN(69),CON(81)_vs_SCN(69),CON(37)_vs_SCN(69),CON(67)_vs_SCN(69),CON(38)_vs_SCN(69),CON(83)_vs_SCN(69),DMN(32)_vs_SCN(69),DMN(40)_vs_SCN(69),DMN(23)_vs_SCN(69),DMN(71)_vs_SCN(69),DMN(17)_vs_SCN(69),DMN(51)_vs_SCN(69),DMN(94)_vs_SCN(69),CBN(13)_vs_SCN(69),CBN(18)_vs_SCN(69),CBN(4)_vs_SCN(69),CBN(7)_vs_SCN(69),SCN(98)_vs_SCN(53),SCN(99)_vs_SCN(53),SCN(45)_vs_SCN(53),ADN(21)_vs_SCN(53),ADN(56)_vs_SCN(53),SMN(3)_vs_SCN(53),SMN(9)_vs_SCN(53),SMN(2)_vs_SCN(53),SMN(11)_vs_SCN(53),SMN(27)_vs_SCN(53),SMN(54)_vs_SCN(53),SMN(66)_vs_SCN(53),SMN(80)_vs_SCN(53),SMN(72)_vs_SCN(53),VSN(16)_vs_SCN(53),VSN(5)_vs_SCN(53),VSN(62)_vs_SCN(53),VSN(15)_vs_SCN(53),VSN(12)_vs_SCN(53),VSN(93)_vs_SCN(53),VSN(20)_vs_SCN(53),VSN(8)_vs_SCN(53),VSN(77)_vs_SCN(53),CON(68)_vs_SCN(53),CON(33)_vs_SCN(53),CON(43)_vs_SCN(53),CON(70)_vs_SCN(53),CON(61)_vs_SCN(53),CON(55)_vs_SCN(53),CON(63)_vs_SCN(53),CON(79)_vs_SCN(53),CON(84)_vs_SCN(53),CON(96)_vs_SCN(53),CON(88)_vs_SCN(53),CON(48)_vs_SCN(53),CON(81)_vs_SCN(53),CON(37)_vs_SCN(53),CON(67)_vs_SCN(53),CON(38)_vs_SCN(53),CON(83)_vs_SCN(53),DMN(32)_vs_SCN(53),DMN(40)_vs_SCN(53),DMN(23)_vs_SCN(53),DMN(71)_vs_SCN(53),DMN(17)_vs_SCN(53),DMN(51)_vs_SCN(53),DMN(94)_vs_SCN(53),CBN(13)_vs_SCN(53),CBN(18)_vs_SCN(53),CBN(4)_vs_SCN(53),CBN(7)_vs_SCN(53),SCN(99)_vs_SCN(98),SCN(45)_vs_SCN(98),ADN(21)_vs_SCN(98),ADN(56)_vs_SCN(98),SMN(3)_vs_SCN(98),SMN(9)_vs_SCN(98),SMN(2)_vs_SCN(98),SMN(11)_vs_SCN(98),SMN(27)_vs_SCN(98),SMN(54)_vs_SCN(98),SMN(66)_vs_SCN(98),SMN(80)_vs_SCN(98),SMN(72)_vs_SCN(98),VSN(16)_vs_SCN(98),VSN(5)_vs_SCN(98),VSN(62)_vs_SCN(98),VSN(15)_vs_SCN(98),VSN(12)_vs_SCN(98),VSN(93)_vs_SCN(98),VSN(20)_vs_SCN(98),VSN(8)_vs_SCN(98),VSN(77)_vs_SCN(98),CON(68)_vs_SCN(98),CON(33)_vs_SCN(98),CON(43)_vs_SCN(98),CON(70)_vs_SCN(98),CON(61)_vs_SCN(98),CON(55)_vs_SCN(98),CON(63)_vs_SCN(98),CON(79)_vs_SCN(98),CON(84)_vs_SCN(98),CON(96)_vs_SCN(98),CON(88)_vs_SCN(98),CON(48)_vs_SCN(98),CON(81)_vs_SCN(98),CON(37)_vs_SCN(98),CON(67)_vs_SCN(98),CON(38)_vs_SCN(98),CON(83)_vs_SCN(98),DMN(32)_vs_SCN(98),DMN(40)_vs_SCN(98),DMN(23)_vs_SCN(98),DMN(71)_vs_SCN(98),DMN(17)_vs_SCN(98),DMN(51)_vs_SCN(98),DMN(94)_vs_SCN(98),CBN(13)_vs_SCN(98),CBN(18)_vs_SCN(98),CBN(4)_vs_SCN(98),CBN(7)_vs_SCN(98),SCN(45)_vs_SCN(99),ADN(21)_vs_SCN(99),ADN(56)_vs_SCN(99),SMN(3)_vs_SCN(99),SMN(9)_vs_SCN(99),SMN(2)_vs_SCN(99),SMN(11)_vs_SCN(99),SMN(27)_vs_SCN(99),SMN(54)_vs_SCN(99),SMN(66)_vs_SCN(99),SMN(80)_vs_SCN(99),SMN(72)_vs_SCN(99),VSN(16)_vs_SCN(99),VSN(5)_vs_SCN(99),VSN(62)_vs_SCN(99),VSN(15)_vs_SCN(99),VSN(12)_vs_SCN(99),VSN(93)_vs_SCN(99),VSN(20)_vs_SCN(99),VSN(8)_vs_SCN(99),VSN(77)_vs_SCN(99),CON(68)_vs_SCN(99),CON(33)_vs_SCN(99),CON(43)_vs_SCN(99),CON(70)_vs_SCN(99),CON(61)_vs_SCN(99),CON(55)_vs_SCN(99),CON(63)_vs_SCN(99),CON(79)_vs_SCN(99),CON(84)_vs_SCN(99),CON(96)_vs_SCN(99),CON(88)_vs_SCN(99),CON(48)_vs_SCN(99),CON(81)_vs_SCN(99),CON(37)_vs_SCN(99),CON(67)_vs_SCN(99),CON(38)_vs_SCN(99),CON(83)_vs_SCN(99),DMN(32)_vs_SCN(99),DMN(40)_vs_SCN(99),DMN(23)_vs_SCN(99),DMN(71)_vs_SCN(99),DMN(17)_vs_SCN(99),DMN(51)_vs_SCN(99),DMN(94)_vs_SCN(99),CBN(13)_vs_SCN(99),CBN(18)_vs_SCN(99),CBN(4)_vs_SCN(99),CBN(7)_vs_SCN(99),ADN(21)_vs_SCN(45),ADN(56)_vs_SCN(45),SMN(3)_vs_SCN(45),SMN(9)_vs_SCN(45),SMN(2)_vs_SCN(45),SMN(11)_vs_SCN(45),SMN(27)_vs_SCN(45),SMN(54)_vs_SCN(45),SMN(66)_vs_SCN(45),SMN(80)_vs_SCN(45)

In [15]:
def make_train(data, train_scores):
      return pd.merge(train_scores, data, on='Id', how = 'left')  # Данные для обучения (Иксы и игреки)    return 

In [16]:
#data = pd.merge(fnc, loading, on='Id')

In [17]:
train_data = make_train(data, train_scores)

In [18]:
#df.drop(['B', 'C'], axis=1)

In [19]:
train_data.head()

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),SMN(27)_vs_SCN(69),SMN(54)_vs_SCN(69),SMN(66)_vs_SCN(69),SMN(80)_vs_SCN(69),SMN(72)_vs_SCN(69),VSN(16)_vs_SCN(69),VSN(5)_vs_SCN(69),VSN(62)_vs_SCN(69),VSN(15)_vs_SCN(69),VSN(12)_vs_SCN(69),VSN(93)_vs_SCN(69),VSN(20)_vs_SCN(69),VSN(8)_vs_SCN(69),VSN(77)_vs_SCN(69),CON(68)_vs_SCN(69),CON(33)_vs_SCN(69),CON(43)_vs_SCN(69),CON(70)_vs_SCN(69),CON(61)_vs_SCN(69),CON(55)_vs_SCN(69),CON(63)_vs_SCN(69),CON(79)_vs_SCN(69),CON(84)_vs_SCN(69),CON(96)_vs_SCN(69),CON(88)_vs_SCN(69),CON(48)_vs_SCN(69),CON(81)_vs_SCN(69),CON(37)_vs_SCN(69),CON(67)_vs_SCN(69),CON(38)_vs_SCN(69),CON(83)_vs_SCN(69),DMN(32)_vs_SCN(69),DMN(40)_vs_SCN(69),DMN(23)_vs_SCN(69),DMN(71)_vs_SCN(69),DMN(17)_vs_SCN(69),DMN(51)_vs_SCN(69),DMN(94)_vs_SCN(69),CBN(13)_vs_SCN(69),CBN(18)_vs_SCN(69),CBN(4)_vs_SCN(69),CBN(7)_vs_SCN(69),SCN(98)_vs_SCN(53),SCN(99)_vs_SCN(53),SCN(45)_vs_SCN(53),ADN(21)_vs_SCN(53),ADN(56)_vs_SCN(53),SMN(3)_vs_SCN(53),SMN(9)_vs_SCN(53),SMN(2)_vs_SCN(53),SMN(11)_vs_SCN(53),SMN(27)_vs_SCN(53),SMN(54)_vs_SCN(53),SMN(66)_vs_SCN(53),SMN(80)_vs_SCN(53),SMN(72)_vs_SCN(53),VSN(16)_vs_SCN(53),VSN(5)_vs_SCN(53),VSN(62)_vs_SCN(53),VSN(15)_vs_SCN(53),VSN(12)_vs_SCN(53),VSN(93)_vs_SCN(53),VSN(20)_vs_SCN(53),VSN(8)_vs_SCN(53),VSN(77)_vs_SCN(53),CON(68)_vs_SCN(53),CON(33)_vs_SCN(53),CON(43)_vs_SCN(53),CON(70)_vs_SCN(53),CON(61)_vs_SCN(53),CON(55)_vs_SCN(53),CON(63)_vs_SCN(53),CON(79)_vs_SCN(53),CON(84)_vs_SCN(53),CON(96)_vs_SCN(53),CON(88)_vs_SCN(53),CON(48)_vs_SCN(53),CON(81)_vs_SCN(53),CON(37)_vs_SCN(53),CON(67)_vs_SCN(53),CON(38)_vs_SCN(53),CON(83)_vs_SCN(53),DMN(32)_vs_SCN(53),DMN(40)_vs_SCN(53),DMN(23)_vs_SCN(53),DMN(71)_vs_SCN(53),DMN(17)_vs_SCN(53),DMN(51)_vs_SCN(53),DMN(94)_vs_SCN(53),CBN(13)_vs_SCN(53),CBN(18)_vs_SCN(53),CBN(4)_vs_SCN(53),CBN(7)_vs_SCN(53),SCN(99)_vs_SCN(98),SCN(45)_vs_SCN(98),ADN(21)_vs_SCN(98),ADN(56)_vs_SCN(98),SMN(3)_vs_SCN(98),SMN(9)_vs_SCN(98),SMN(2)_vs_SCN(98),SMN(11)_vs_SCN(98),SMN(27)_vs_SCN(98),SMN(54)_vs_SCN(98),SMN(66)_vs_SCN(98),SMN(80)_vs_SCN(98),SMN(72)_vs_SCN(98),VSN(16)_vs_SCN(98),VSN(5)_vs_SCN(98),VSN(62)_vs_SCN(98),VSN(15)_vs_SCN(98),VSN(12)_vs_SCN(98),VSN(93)_vs_SCN(98),VSN(20)_vs_SCN(98),VSN(8)_vs_SCN(98),VSN(77)_vs_SCN(98),CON(68)_vs_SCN(98),CON(33)_vs_SCN(98),CON(43)_vs_SCN(98),CON(70)_vs_SCN(98),CON(61)_vs_SCN(98),CON(55)_vs_SCN(98),CON(63)_vs_SCN(98),CON(79)_vs_SCN(98),CON(84)_vs_SCN(98),CON(96)_vs_SCN(98),CON(88)_vs_SCN(98),CON(48)_vs_SCN(98),CON(81)_vs_SCN(98),CON(37)_vs_SCN(98),CON(67)_vs_SCN(98),CON(38)_vs_SCN(98),CON(83)_vs_SCN(98),DMN(32)_vs_SCN(98),DMN(40)_vs_SCN(98),DMN(23)_vs_SCN(98),DMN(71)_vs_SCN(98),DMN(17)_vs_SCN(98),DMN(51)_vs_SCN(98),DMN(94)_vs_SCN(98),CBN(13)_vs_SCN(98),CBN(18)_vs_SCN(98),CBN(4)_vs_SCN(98),CBN(7)_vs_SCN(98),SCN(45)_vs_SCN(99),ADN(21)_vs_SCN(99),ADN(56)_vs_SCN(99),SMN(3)_vs_SCN(99),SMN(9)_vs_SCN(99),SMN(2)_vs_SCN(99),SMN(11)_vs_SCN(99),SMN(27)_vs_SCN(99),SMN(54)_vs_SCN(99),SMN(66)_vs_SCN(99),SMN(80)_vs_SCN(99),SMN(72)_vs_SCN(99),VSN(16)_vs_SCN(99),VSN(5)_vs_SCN(99),VSN(62)_vs_SCN(99),VSN(15)_vs_SCN(99),VSN(12)_vs_SCN(99),VSN(93)_vs_SCN(99),VSN(20)_vs_SCN(99),VSN(8)_vs_SCN(99),VSN(77)_vs_SCN(99),CON(68)_vs_SCN(99),CON(33)_vs_SCN(99),CON(43)_vs_SCN(99),CON(70)_vs_SCN(99),CON(61)_vs_SCN(99),CON(55)_vs_SCN(99),CON(63)_vs_SCN(99),CON(79)_vs_SCN(99),CON(84)_vs_SCN(99),CON(96)_vs_SCN(99),CON(88)_vs_SCN(99),CON(48)_vs_SCN(99),CON(81)_vs_SCN(99),CON(37)_vs_SCN(99),CON(67)_vs_SCN(99),CON(38)_vs_SCN(99),CON(83)_vs_SCN(99),DMN(32)_vs_SCN(99),DMN(40)_vs_SCN(99),DMN(23)_vs_SCN(99),DMN(71)_vs_SCN(99),DMN(17)_vs_SCN(99),DMN(51)_vs_SCN(99),DMN(94)_vs_SCN(99),CBN(13)_vs_SCN(99),CBN(18)_vs_SCN(99),CBN(4)_vs_SCN(99),CBN(7)_vs_SCN(99),ADN(21)_vs_SCN(45),ADN(56)_vs_SCN(45),SMN(3)_vs_SCN(45),SMN(9)_vs_SCN(45),SMN(2)_vs_SCN(45),SMN(11)_vs_SCN(45),SMN(27)_vs_SCN(45),

In [20]:
X_train

NameError: name 'X_train' is not defined

In [21]:
#test?
# generate id
# 

In [22]:
data

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),SMN(27)_vs_SCN(69),SMN(54)_vs_SCN(69),SMN(66)_vs_SCN(69),SMN(80)_vs_SCN(69),SMN(72)_vs_SCN(69),VSN(16)_vs_SCN(69),VSN(5)_vs_SCN(69),VSN(62)_vs_SCN(69),VSN(15)_vs_SCN(69),VSN(12)_vs_SCN(69),VSN(93)_vs_SCN(69),VSN(20)_vs_SCN(69),VSN(8)_vs_SCN(69),VSN(77)_vs_SCN(69),CON(68)_vs_SCN(69),CON(33)_vs_SCN(69),CON(43)_vs_SCN(69),CON(70)_vs_SCN(69),CON(61)_vs_SCN(69),CON(55)_vs_SCN(69),CON(63)_vs_SCN(69),CON(79)_vs_SCN(69),CON(84)_vs_SCN(69),CON(96)_vs_SCN(69),CON(88)_vs_SCN(69),CON(48)_vs_SCN(69),CON(81)_vs_SCN(69),CON(37)_vs_SCN(69),CON(67)_vs_SCN(69),CON(38)_vs_SCN(69),CON(83)_vs_SCN(69),DMN(32)_vs_SCN(69),DMN(40)_vs_SCN(69),DMN(23)_vs_SCN(69),DMN(71)_vs_SCN(69),DMN(17)_vs_SCN(69),DMN(51)_vs_SCN(69),DMN(94)_vs_SCN(69),CBN(13)_vs_SCN(69),CBN(18)_vs_SCN(69),CBN(4)_vs_SCN(69),CBN(7)_vs_SCN(69),SCN(98)_vs_SCN(53),SCN(99)_vs_SCN(53),SCN(45)_vs_SCN(53),ADN(21)_vs_SCN(53),ADN(56)_vs_SCN(53),SMN(3)_vs_SCN(53),SMN(9)_vs_SCN(53),SMN(2)_vs_SCN(53),SMN(11)_vs_SCN(53),SMN(27)_vs_SCN(53),SMN(54)_vs_SCN(53),SMN(66)_vs_SCN(53),SMN(80)_vs_SCN(53),SMN(72)_vs_SCN(53),VSN(16)_vs_SCN(53),VSN(5)_vs_SCN(53),VSN(62)_vs_SCN(53),VSN(15)_vs_SCN(53),VSN(12)_vs_SCN(53),VSN(93)_vs_SCN(53),VSN(20)_vs_SCN(53),VSN(8)_vs_SCN(53),VSN(77)_vs_SCN(53),CON(68)_vs_SCN(53),CON(33)_vs_SCN(53),CON(43)_vs_SCN(53),CON(70)_vs_SCN(53),CON(61)_vs_SCN(53),CON(55)_vs_SCN(53),CON(63)_vs_SCN(53),CON(79)_vs_SCN(53),CON(84)_vs_SCN(53),CON(96)_vs_SCN(53),CON(88)_vs_SCN(53),CON(48)_vs_SCN(53),CON(81)_vs_SCN(53),CON(37)_vs_SCN(53),CON(67)_vs_SCN(53),CON(38)_vs_SCN(53),CON(83)_vs_SCN(53),DMN(32)_vs_SCN(53),DMN(40)_vs_SCN(53),DMN(23)_vs_SCN(53),DMN(71)_vs_SCN(53),DMN(17)_vs_SCN(53),DMN(51)_vs_SCN(53),DMN(94)_vs_SCN(53),CBN(13)_vs_SCN(53),CBN(18)_vs_SCN(53),CBN(4)_vs_SCN(53),CBN(7)_vs_SCN(53),SCN(99)_vs_SCN(98),SCN(45)_vs_SCN(98),ADN(21)_vs_SCN(98),ADN(56)_vs_SCN(98),SMN(3)_vs_SCN(98),SMN(9)_vs_SCN(98),SMN(2)_vs_SCN(98),SMN(11)_vs_SCN(98),SMN(27)_vs_SCN(98),SMN(54)_vs_SCN(98),SMN(66)_vs_SCN(98),SMN(80)_vs_SCN(98),SMN(72)_vs_SCN(98),VSN(16)_vs_SCN(98),VSN(5)_vs_SCN(98),VSN(62)_vs_SCN(98),VSN(15)_vs_SCN(98),VSN(12)_vs_SCN(98),VSN(93)_vs_SCN(98),VSN(20)_vs_SCN(98),VSN(8)_vs_SCN(98),VSN(77)_vs_SCN(98),CON(68)_vs_SCN(98),CON(33)_vs_SCN(98),CON(43)_vs_SCN(98),CON(70)_vs_SCN(98),CON(61)_vs_SCN(98),CON(55)_vs_SCN(98),CON(63)_vs_SCN(98),CON(79)_vs_SCN(98),CON(84)_vs_SCN(98),CON(96)_vs_SCN(98),CON(88)_vs_SCN(98),CON(48)_vs_SCN(98),CON(81)_vs_SCN(98),CON(37)_vs_SCN(98),CON(67)_vs_SCN(98),CON(38)_vs_SCN(98),CON(83)_vs_SCN(98),DMN(32)_vs_SCN(98),DMN(40)_vs_SCN(98),DMN(23)_vs_SCN(98),DMN(71)_vs_SCN(98),DMN(17)_vs_SCN(98),DMN(51)_vs_SCN(98),DMN(94)_vs_SCN(98),CBN(13)_vs_SCN(98),CBN(18)_vs_SCN(98),CBN(4)_vs_SCN(98),CBN(7)_vs_SCN(98),SCN(45)_vs_SCN(99),ADN(21)_vs_SCN(99),ADN(56)_vs_SCN(99),SMN(3)_vs_SCN(99),SMN(9)_vs_SCN(99),SMN(2)_vs_SCN(99),SMN(11)_vs_SCN(99),SMN(27)_vs_SCN(99),SMN(54)_vs_SCN(99),SMN(66)_vs_SCN(99),SMN(80)_vs_SCN(99),SMN(72)_vs_SCN(99),VSN(16)_vs_SCN(99),VSN(5)_vs_SCN(99),VSN(62)_vs_SCN(99),VSN(15)_vs_SCN(99),VSN(12)_vs_SCN(99),VSN(93)_vs_SCN(99),VSN(20)_vs_SCN(99),VSN(8)_vs_SCN(99),VSN(77)_vs_SCN(99),CON(68)_vs_SCN(99),CON(33)_vs_SCN(99),CON(43)_vs_SCN(99),CON(70)_vs_SCN(99),CON(61)_vs_SCN(99),CON(55)_vs_SCN(99),CON(63)_vs_SCN(99),CON(79)_vs_SCN(99),CON(84)_vs_SCN(99),CON(96)_vs_SCN(99),CON(88)_vs_SCN(99),CON(48)_vs_SCN(99),CON(81)_vs_SCN(99),CON(37)_vs_SCN(99),CON(67)_vs_SCN(99),CON(38)_vs_SCN(99),CON(83)_vs_SCN(99),DMN(32)_vs_SCN(99),DMN(40)_vs_SCN(99),DMN(23)_vs_SCN(99),DMN(71)_vs_SCN(99),DMN(17)_vs_SCN(99),DMN(51)_vs_SCN(99),DMN(94)_vs_SCN(99),CBN(13)_vs_SCN(99),CBN(18)_vs_SCN(99),CBN(4)_vs_SCN(99),CBN(7)_vs_SCN(99),ADN(21)_vs_SCN(45),ADN(56)_vs_SCN(45),SMN(3)_vs_SCN(45),SMN(9)_vs_SCN(45),SMN(2)_vs_SCN(45),SMN(11)_vs_SCN(45),SMN(27)_vs_SCN(45),SMN(54)_vs_SCN(45),SMN(66)_vs_SCN(45),SMN(80)_vs_SCN(45)

In [23]:
def get_x_y(data, target_name, features_list):
    """
    что делает эта функция
    """
    X  = data[list(features_list)]
    if target_name!=None:
        y = data[target_name].fillna(np.mean(data[target_name]))
    else:
        y = None
    #prepare X, y
    # make crossval
    #find average metric
    #kf = KFold(n_splits=5)
    #kf.get_n_splits(X)
    return X, y

In [24]:
X

NameError: name 'X' is not defined

In [86]:
y

NameError: name 'y' is not defined

In [25]:
features1 = list(X.columns[:])

NameError: name 'X' is not defined

In [26]:
target_name = 'domain1_var1'

In [84]:
#X, y = get_x_y(train_data, target_name , features1)

In [85]:
# оч важно
X_sub, y_sub  = get_x_y(test_data, None, features1)   
X = X_sub
y = y_sub

NameError: name 'features1' is not defined

In [29]:
y.head()

NameError: name 'y' is not defined

In [30]:
X.head()

NameError: name 'X' is not defined

In [31]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)

def main_model(test_data, target_name, features1):
    X_sub, y_sub  = get_x_y(test_data, target_name, features1)  
    for train_index, test_index in kf.split(X_sub):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.loc[train_index, features1], X.loc[test_index, features1]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model_for_test =  LGBMRegressor(n_estimators = 200, max_depth=3)
        model_for_test.fit(X,y)
        test_prediction = model_for_test.predict(X_test)
        mae = mean_absolute_error(y_true=y_test, y_pred=prediction)
        average_pred = np.mean(prediction)
        metric = mae /average_pred
        print('mae:', mae)
        print("metric:", metric)
    return test_prediction, mae, average_pred, metric


NameError: name 'X' is not defined

In [32]:


k  = KFold(n_splits=5)
kf.get_n_splits(X)

print(kf)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.loc[train_index, features1], X.loc[test_index, features1]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    model = LGBMRegressor(n_estimators = 200, max_depth=3)
    model.fit(X_train,y_train )
    prediction = model.predict(X_test)
    mae = mean_absolute_error(y_true=y_test, y_pred=prediction)
    average_pred = np.mean(prediction)
    metric = mae /average_pred
    print('mae:', mae)
    print("metric:", metric)

    
    # predict on test 
    

    #plot feature importance
    
#todo find average metric for each fold

NameError: name 'X' is not defined

In [33]:
model_for_test =  LGBMRegressor(n_estimators = 200, max_depth=3)

model_for_test.fit(X,y)

test_prediction = model_for_test.predict(X_sub)

NameError: name 'X' is not defined

In [34]:
test_prediction[0]

NameError: name 'test_prediction' is not defined

In [35]:
len(test_prediction)

NameError: name 'test_prediction' is not defined

In [36]:
test_data

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),SMN(27)_vs_SCN(69),SMN(54)_vs_SCN(69),SMN(66)_vs_SCN(69),SMN(80)_vs_SCN(69),SMN(72)_vs_SCN(69),VSN(16)_vs_SCN(69),VSN(5)_vs_SCN(69),VSN(62)_vs_SCN(69),VSN(15)_vs_SCN(69),VSN(12)_vs_SCN(69),VSN(93)_vs_SCN(69),VSN(20)_vs_SCN(69),VSN(8)_vs_SCN(69),VSN(77)_vs_SCN(69),CON(68)_vs_SCN(69),CON(33)_vs_SCN(69),CON(43)_vs_SCN(69),CON(70)_vs_SCN(69),CON(61)_vs_SCN(69),CON(55)_vs_SCN(69),CON(63)_vs_SCN(69),CON(79)_vs_SCN(69),CON(84)_vs_SCN(69),CON(96)_vs_SCN(69),CON(88)_vs_SCN(69),CON(48)_vs_SCN(69),CON(81)_vs_SCN(69),CON(37)_vs_SCN(69),CON(67)_vs_SCN(69),CON(38)_vs_SCN(69),CON(83)_vs_SCN(69),DMN(32)_vs_SCN(69),DMN(40)_vs_SCN(69),DMN(23)_vs_SCN(69),DMN(71)_vs_SCN(69),DMN(17)_vs_SCN(69),DMN(51)_vs_SCN(69),DMN(94)_vs_SCN(69),CBN(13)_vs_SCN(69),CBN(18)_vs_SCN(69),CBN(4)_vs_SCN(69),CBN(7)_vs_SCN(69),SCN(98)_vs_SCN(53),SCN(99)_vs_SCN(53),SCN(45)_vs_SCN(53),ADN(21)_vs_SCN(53),ADN(56)_vs_SCN(53),SMN(3)_vs_SCN(53),SMN(9)_vs_SCN(53),SMN(2)_vs_SCN(53),SMN(11)_vs_SCN(53),SMN(27)_vs_SCN(53),SMN(54)_vs_SCN(53),SMN(66)_vs_SCN(53),SMN(80)_vs_SCN(53),SMN(72)_vs_SCN(53),VSN(16)_vs_SCN(53),VSN(5)_vs_SCN(53),VSN(62)_vs_SCN(53),VSN(15)_vs_SCN(53),VSN(12)_vs_SCN(53),VSN(93)_vs_SCN(53),VSN(20)_vs_SCN(53),VSN(8)_vs_SCN(53),VSN(77)_vs_SCN(53),CON(68)_vs_SCN(53),CON(33)_vs_SCN(53),CON(43)_vs_SCN(53),CON(70)_vs_SCN(53),CON(61)_vs_SCN(53),CON(55)_vs_SCN(53),CON(63)_vs_SCN(53),CON(79)_vs_SCN(53),CON(84)_vs_SCN(53),CON(96)_vs_SCN(53),CON(88)_vs_SCN(53),CON(48)_vs_SCN(53),CON(81)_vs_SCN(53),CON(37)_vs_SCN(53),CON(67)_vs_SCN(53),CON(38)_vs_SCN(53),CON(83)_vs_SCN(53),DMN(32)_vs_SCN(53),DMN(40)_vs_SCN(53),DMN(23)_vs_SCN(53),DMN(71)_vs_SCN(53),DMN(17)_vs_SCN(53),DMN(51)_vs_SCN(53),DMN(94)_vs_SCN(53),CBN(13)_vs_SCN(53),CBN(18)_vs_SCN(53),CBN(4)_vs_SCN(53),CBN(7)_vs_SCN(53),SCN(99)_vs_SCN(98),SCN(45)_vs_SCN(98),ADN(21)_vs_SCN(98),ADN(56)_vs_SCN(98),SMN(3)_vs_SCN(98),SMN(9)_vs_SCN(98),SMN(2)_vs_SCN(98),SMN(11)_vs_SCN(98),SMN(27)_vs_SCN(98),SMN(54)_vs_SCN(98),SMN(66)_vs_SCN(98),SMN(80)_vs_SCN(98),SMN(72)_vs_SCN(98),VSN(16)_vs_SCN(98),VSN(5)_vs_SCN(98),VSN(62)_vs_SCN(98),VSN(15)_vs_SCN(98),VSN(12)_vs_SCN(98),VSN(93)_vs_SCN(98),VSN(20)_vs_SCN(98),VSN(8)_vs_SCN(98),VSN(77)_vs_SCN(98),CON(68)_vs_SCN(98),CON(33)_vs_SCN(98),CON(43)_vs_SCN(98),CON(70)_vs_SCN(98),CON(61)_vs_SCN(98),CON(55)_vs_SCN(98),CON(63)_vs_SCN(98),CON(79)_vs_SCN(98),CON(84)_vs_SCN(98),CON(96)_vs_SCN(98),CON(88)_vs_SCN(98),CON(48)_vs_SCN(98),CON(81)_vs_SCN(98),CON(37)_vs_SCN(98),CON(67)_vs_SCN(98),CON(38)_vs_SCN(98),CON(83)_vs_SCN(98),DMN(32)_vs_SCN(98),DMN(40)_vs_SCN(98),DMN(23)_vs_SCN(98),DMN(71)_vs_SCN(98),DMN(17)_vs_SCN(98),DMN(51)_vs_SCN(98),DMN(94)_vs_SCN(98),CBN(13)_vs_SCN(98),CBN(18)_vs_SCN(98),CBN(4)_vs_SCN(98),CBN(7)_vs_SCN(98),SCN(45)_vs_SCN(99),ADN(21)_vs_SCN(99),ADN(56)_vs_SCN(99),SMN(3)_vs_SCN(99),SMN(9)_vs_SCN(99),SMN(2)_vs_SCN(99),SMN(11)_vs_SCN(99),SMN(27)_vs_SCN(99),SMN(54)_vs_SCN(99),SMN(66)_vs_SCN(99),SMN(80)_vs_SCN(99),SMN(72)_vs_SCN(99),VSN(16)_vs_SCN(99),VSN(5)_vs_SCN(99),VSN(62)_vs_SCN(99),VSN(15)_vs_SCN(99),VSN(12)_vs_SCN(99),VSN(93)_vs_SCN(99),VSN(20)_vs_SCN(99),VSN(8)_vs_SCN(99),VSN(77)_vs_SCN(99),CON(68)_vs_SCN(99),CON(33)_vs_SCN(99),CON(43)_vs_SCN(99),CON(70)_vs_SCN(99),CON(61)_vs_SCN(99),CON(55)_vs_SCN(99),CON(63)_vs_SCN(99),CON(79)_vs_SCN(99),CON(84)_vs_SCN(99),CON(96)_vs_SCN(99),CON(88)_vs_SCN(99),CON(48)_vs_SCN(99),CON(81)_vs_SCN(99),CON(37)_vs_SCN(99),CON(67)_vs_SCN(99),CON(38)_vs_SCN(99),CON(83)_vs_SCN(99),DMN(32)_vs_SCN(99),DMN(40)_vs_SCN(99),DMN(23)_vs_SCN(99),DMN(71)_vs_SCN(99),DMN(17)_vs_SCN(99),DMN(51)_vs_SCN(99),DMN(94)_vs_SCN(99),CBN(13)_vs_SCN(99),CBN(18)_vs_SCN(99),CBN(4)_vs_SCN(99),CBN(7)_vs_SCN(99),ADN(21)_vs_SCN(45),ADN(56)_vs_SCN(45),SMN(3)_vs_SCN(45),SMN(9)_vs_SCN(45),SMN(2)_vs_SCN(45),SMN(11)_vs_SCN(45),SMN(27)_vs_SCN(45),SMN(54)_vs_SCN(45),SMN(66)_vs_SCN(45),SMN(80)_vs_SCN(45)

In [37]:
test_data[target_name] =test_prediction

NameError: name 'test_prediction' is not defined

In [38]:
test_id 

,Id
0,10003
1,10006
2,10010
3,10011
4,10012
...,...
5872,21745
5873,21748
5874,21749
5875,21751


In [93]:
def data_frame_with_answers():
    df_with_answers = pd.DataFrame(test_id['Id'])
    #df_with_answers = pd.DataFrame({'Id': [], 'age': [] , 'domain1_var1': [], 'domain1_var2': [], 'domain1_var2': [], 'domain2_var1': [], 'domain2_var2': []})
    df_with_answers['age'] = df_with_answers.apply(lambda _: 0, axis=1)
    df_with_answers['domain1_var1'] = df_with_answers.apply(lambda _: 0, axis=1)
    df_with_answers['domain1_var2'] = df_with_answers.apply(lambda _: 0, axis=1)
    df_with_answers['domain2_var1'] = df_with_answers.apply(lambda _: 0, axis=1)
    df_with_answers['domain2_var2'] = df_with_answers.apply(lambda _: 0, axis=1)
    #df['foo'] = df.apply(lambda _: '', axis=1)
    return df_with_answers

In [94]:
df_with_answers = data_frame_with_answers()

In [95]:
df_with_answers

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10003,0,0,0,0,0
1,10006,0,0,0,0,0
2,10010,0,0,0,0,0
3,10011,0,0,0,0,0
4,10012,0,0,0,0,0
...,...,...,...,...,...,...
5872,21745,0,0,0,0,0
5873,21748,0,0,0,0,0
5874,21749,0,0,0,0,0
5875,21751,0,0,0,0,0


In [121]:
print(df_with_answers.loc[5, 'domain1_var1'])


0


In [125]:
x, y = [] ,[]
for i in ['age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2']:
    x +=  list(df_with_answers['Id'].apply(lambda x: str(x)+'_'+i))
    y +=  list(df_with_answers[i])    
dfansw = pd.DataFrame({'Id': x, 'Predicted': y})

In [126]:
dfansw


,Id,Predicted
0,10003_age,0
1,10006_age,0
2,10010_age,0
3,10011_age,0
4,10012_age,0
...,...,...
29380,21745_domain2_var2,0
29381,21748_domain2_var2,0
29382,21749_domain2_var2,0
29383,21751_domain2_var2,0


In [118]:
def transform_data_in_ss():
    dfansw = pd.DataFrame({'Id': [], 'Predicted': []})
    for i in range(0, 5*len(df_with_answers), 5):
        Id_content0 = str(df_with_answers.loc[i, 'Id'])+'_'+str('Age')
        Id_content1 = str(df_with_answers.loc[i, 'Id'])+'_'+str('domain1_var1')
        Id_content2 = str(df_with_answers.loc[i, 'Id'])+'_'+str('domain1_var2')
        Id_content3 = str(df_with_answers.loc[i, 'Id'])+'_'+str('domain2_var1')
        Id_content4 = str(df_with_answers.loc[i, 'Id'])+'_'+str('domain2_var2')
        #  херня
        dfansw.loc[i] = [Id_content0, df_with_answers.loc[i, 'Age']]
        dfansw.loc[i+1] = [Id_content1, df_with_answers.loc[i, 'domain1_var1']]
        dfansw.loc[i+2] = [Id_content2, df_with_answers.loc[i, 'domain1_var2']]
        dfansw.loc[i+3] = [Id_content3, df_with_answers.loc[i, 'domain2_var1']]
        dfansw.loc[i+4] = [Id_content4, df_with_answers.loc[i, 'domain2_var2']]
    return dfansw
        

In [119]:
kek = transform_data_in_ss()

KeyError: 'Age'

In [120]:
kek

<function __main__.transform_data_in_ss()>

In [40]:
ss.head(10)

,Id,Predicted
0,10003_age,50.0
1,10003_domain1_var1,50.0
2,10003_domain1_var2,50.0
3,10003_domain2_var1,50.0
4,10003_domain2_var2,50.0
5,10006_age,50.0
6,10006_domain1_var1,50.0
7,10006_domain1_var2,50.0
8,10006_domain2_var1,50.0
9,10006_domain2_var2,50.0


In [94]:
def generate_submission_file(target_name, test_prediction):
    df1 = pd.DataFrame({'Id': [], 'Predicted': []})
    for i in range(0, len(test_prediction)):
        Id_content = str(i)+'_'+str(target_name)
        df1.loc[i] = [Id_content, test_prediction[i]]
    return df1

In [95]:
a = generate_submission_file(target_name, test_prediction)

In [96]:
a

,Id,Predicted
0,0_domain1_var1,49.743974
1,1_domain1_var1,53.886369
2,2_domain1_var1,48.747336
3,3_domain1_var1,53.504879
4,4_domain1_var1,55.681883
...,...,...
11749,11749_domain1_var1,50.784660
11750,11750_domain1_var1,47.473910
11751,11751_domain1_var1,53.619625
11752,11752_domain1_var1,45.106371


In [48]:
print('Feature importances:', list(model.feature_importances_))

Feature importances: [58, 55, 54, 38, 66, 83, 47, 56, 51, 57, 58, 59, 74, 42, 66, 82, 43, 56, 90, 65, 58]


In [50]:
features1

['IC_06',
 'IC_10',
 'IC_09',
 'IC_18',
 'IC_04',
 'IC_12',
 'IC_24',
 'IC_15',
 'IC_13',
 'IC_17',
 'IC_02',
 'IC_08',
 'IC_03',
 'IC_21',
 'IC_28',
 'IC_11',
 'IC_20',
 'IC_30',
 'IC_22',
 'IC_29',
 'IC_14']

In [18]:
y_train = kek.iloc[:,1]

In [19]:
y_train

0       57.436077
1       59.580851
2       71.413018
3       66.532630
4       38.617381
          ...    
5872    14.257265
5873    55.456978
5874    48.948756
5875    66.532630
5876    68.820928
Name: age, Length: 5877, dtype: float64

In [11]:
X_test = 

,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),...,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,0.368580,0.166876,0.438148,0.341007,-0.186251,0.049096,0.121417,-0.174268,-0.231578,0.000947,...,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,0.151696,-0.024819,0.217504,0.418072,-0.227234,-0.064052,-0.143832,-0.118116,-0.054825,0.038732,...,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
2,0.343415,0.109974,0.741641,0.578558,-0.676446,-0.436960,-0.295663,-0.377790,-0.344963,-0.294511,...,0.019565,0.030616,0.018184,-0.010469,0.029799,0.015435,0.005211,-0.028882,0.031427,0.018164
3,0.132793,0.258255,0.490769,0.342717,0.091112,0.107969,0.029220,-0.026237,0.094742,0.098802,...,0.002658,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760
4,0.291921,0.251254,0.416470,0.511719,-0.362626,-0.164710,-0.289059,-0.015537,-0.087316,-0.142736,...,0.009702,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,0.227970,-0.222489,0.250417,0.442642,-0.221094,-0.234935,-0.296614,-0.273216,-0.106481,-0.157194,...,0.011454,0.022840,0.006448,-0.007203,0.029689,0.004942,0.007751,-0.020226,0.028821,0.017492
11750,0.455052,0.483856,0.589565,0.633691,0.161995,-0.175318,0.015480,0.135612,0.172296,0.129111,...,0.012303,0.018314,0.012326,-0.012152,0.024617,0.002096,0.001911,-0.008028,0.035093,0.017439
11751,0.118257,0.452123,0.608328,0.422485,-0.106427,-0.008484,-0.010050,-0.122015,0.092994,-0.071942,...,0.012055,0.020741,0.001160,-0.007140,0.025236,0.002026,0.001876,-0.014612,0.021665,0.019592
11752,0.051042,0.088581,0.551354,0.305542,-0.034378,-0.094778,-0.089954,-0.222876,-0.197503,-0.192861,...,0.019220,0.026282,0.020715,-0.008130,0.025149,0.007695,0.003226,-0.003505,0.029326,0.014235


In [ ]:
y_test =

In [ ]:
parameters = {}

In [ ]:
model = lgb.train(parameters, X_train)

In [ ]:
y = model.predict(X_test)

In [ ]:
print(np.mean(cross_val_score(model, X_train, y_train, cv=5)))